In [ ]:
from underthesea import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
import joblib
import pandas as pd
from sklearn.model_selection import train_test_split
import regex as re
from sklearn import svm
emb = None

In [43]:
#Bước 1: Lấy dataset từ 2 file CSV (pos and neg)
def load_data():
    df1 = pd.read_csv("data_train_neg.csv")
    df2=pd.read_csv("data_train_pos.csv")
    frames=[df1,df2]
    resulst=pd.concat(frames)
    return resulst
#  Load data from file
data = load_data()

In [44]:
# Bước 2. Xử lý text
def remove_icon(text):
    pattern = re.compile('[\W_]+')
    dirty_list = text.split()
    clean_list = [pattern.sub('', word) for word in dirty_list]
    clean_string = ' '.join(clean_list)
    return clean_string

def standardize_data(post):
    post=remove_icon(post)
    post=post.replace('.','')
    post=post.replace(',','')
    post=post.replace(' k ',' không ')
    post=post.replace(' mk ',' mình ')
    post=post.replace('\n',' ')
    post=post.replace(' cx ',' cũng ')
    post=post.replace('  ',' ')
    post=post.replace(' r ',' rồi ')
    post=post.replace(' dc ',' được ')
    post=post.strip()
    post = re.sub(r"[\.,\?]+$-", "", post)
    post=word_tokenize(post.lower(),format="text")
    return post
data["Text"] = data.Text.apply(standardize_data)

In [50]:
#Bước 3: Vector hoá các câu trong bộ dữ liệu
def embedding(X_train):
    global  emb
    emb = TfidfVectorizer(min_df=5, max_df=0.8,max_features=3000,sublinear_tf=True)
    emb.fit(X_train)
    X_train =  emb.transform(X_train)
    # Save pkl file
    joblib.dump(emb, 'tfidf.pkl')
    return X_train
    
X_train=data["Text"]
y_train =data["Sentiment"]
# Embeding X_train
X_train = embedding(X_train)

In [52]:
#Bước 4: Train and save model
model = svm.SVC(kernel='linear', C = 1)
model.fit(X_train,y_train)
joblib.dump(model, 'saved_model.pkl')

['saved_model.pkl']